# Recommendation - Data Preparation 🎬

---

<img src="https://cdn-images-1.medium.com/max/1200/0*ePGWILY6GyplT-nn" />

---

In the next few challenges, you will build a powerful **movie recommender**.

We will use the open-source library [LightFM](https://github.com/lyst/lightfm) which provides easy python implementation of **hybrid** recommendation engines.

In this first part, we will prepare the data in order to train efficiently of the model.

We let you load the data `movies` and `ratings` downloaded from the **small** [movielens dataset](https://grouplens.org/datasets/movielens/).



In [10]:

### TODO: Load the movies and ratings datasets
import pandas as pd
movies = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")

print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


**Q1**. What are the different types of recommendation models? Explain briefly with your own words the differences between them.

##Content-based recommendations: 
Recommends movies based on the similairty of the users preferences  

##Ratings-based recommendations: 
Uses a ratings matrix to recommend movies 

##Cluster-based recommendation: 
Uses machine learning to create clusters to recommend movies

**Q1bis**. What data is expected by the LightFM `fit` method? Especially, how does the train data should be organized, and what should be the type of the train dataset? 

It expects a sparse interaction matrix telling us which users have rated which movies it is represented as 1 for rated and 0 for unrated

**Q2**. Explore `movies` and `ratings`, what do those datasets contain? How are they organized?

In [11]:
print(movies.shape)
print(movies.info())
movies.head()



(9742, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
ratings['date_time'] = pd.to_datetime(ratings['timestamp'], unit='s')
print(ratings.shape)
print(ratings.info())
ratings.head()
# ratings.tail()


(100836, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   userId     100836 non-null  int64         
 1   movieId    100836 non-null  int64         
 2   rating     100836 non-null  float64       
 3   timestamp  100836 non-null  int64         
 4   date_time  100836 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 3.8 MB
None


,userId,movieId,rating,timestamp,date_time
0,1,1,4.0,964982703,2000-07-30 18:45:03
1,1,3,4.0,964981247,2000-07-30 18:20:47
2,1,6,4.0,964982224,2000-07-30 18:37:04
3,1,47,5.0,964983815,2000-07-30 19:03:35
4,1,50,5.0,964982931,2000-07-30 18:48:51


---

### Q3 & Q4 are optional
> you can come back to it if you have time after having finished the whole project of the day

We created a few utils functions for you in `utils.py` script. Especially:
- `threshold_interactions_df`:
> Limit interactions df to minimum row and column interactions

**Q3**. Open `src/utils.py` file, and have a look at the documentation of this function to understand its goal and how it works.

Have a look the code to understand fully how it works. You should be familiar with everything.

What does represent the variable `sparsity`? What is the range of values in which sparsity can be?

threshold_interactions_df takes the ineraction matrix and it filters out movies or users that dont have many reviewsT

The sparsity represents how empty (preportion of 0's) in the matrix. The scale is between 0-1.The higher the proportion of the user the the higher the sparsity value. A high sparsity suggest there is lots of data in the dataframe. whereas a lower sparsity closer to 0 repsents little data in the dataframe

taking the number of movie ratings by users then dividing by rows and colums and then multiplied by 100 to make it a percentage. Valus are between 0-100 %

**Q4**. Create a new DataFrame `ratings_thresh`, that filters `ratings` with only:
- users that rated strictly more than 4 movies
- movies that have been rated at least 10 times

How many users/movies remain in this new dataset?

In [18]:
from utils import threshold_interactions_df
print(
    "there was",
    len(ratings.userId.unique()),
    "users and",
    len(ratings.movieId.unique()),
    "movies before performing the threshold"
)

ratings_thresh_df = threshold_interactions_df(
    ratings,
    "userId",
    "movieId",
    5,
    10
)

print(
    "Ther remains",
    len(rating_thresh_df.userId.unique()),
    "users and",
    len(rating_thresh_df.movieId.unique()),
    "movies after performing the threshold"
)


there was 610 users and 9724 movies before performing the threshold
Starting interactions info
Number of rows: 610
Number of cols: 9724
Sparsity: 1.700%
Ending interactions info
Number of rows: 610
Number of columns: 3650
Sparsity: 4.055%


NameError: name 'rating_thresh_df' is not defined

**Q5**. In order to fit a [LightFM](https://lyst.github.io/lightfm/docs/home.html) model, we need to transform our Dataframe to a sparse matrix (cf. below). This is not straightforward so we included the function `df_to_matrix` in `utils.py`.

> 🔦 **Hint**:  Sparse matrices are just **big matrices with a lot of zeros or empty values**.
> 
> Existing tools (Pandas DataFrame, Numpy arrays for example) are not suitable for manipulating this kind of data. So we will use [Scipy sparse matrices](https://docs.scipy.org/doc/scipy-0.14.0/reference/sparse.html).
>
> It exists many different "types" of sparse matrices (CSC, CSR, COO, DIA, etc.). You don't need to know them. Just know that it corresponds to different formats with different methods of manipulation, slicing, indexing, etc.

> 🔦 **Hint 2**:  By going from a DataFrame to a sparse matrix, you will lose the information of the ids (userId and movieId), you will only deal with indices (row number and column number). Therefore, the `df_to_matrix` function also returns dictionaries mapping indexes to ids (ex: uid_to_idx mapping userId to index of the matrix) 


Have a look at the util function documentation, and use it to create 5 new variables:
- a final sparse matrix `ratings_matrix` (this will be the data used to train the model)
- the following utils mappers:
    - `uid_to_idx`
    - `idx_to_uid`
    - `mid_to_idx`
    - `idx_to_mid`

In [20]:
from utils import df_to_matrix

ratings_matrix, uid_to_idx, idx_to_uid, mid_to_idx, idx_to_mid =  df_to_matrix(
    ratings_thresh_df,
    "userId",
    "movieId"
)
ratings_matrix

<610x3650 sparse matrix of type '<class 'numpy.float64'>'
	with 90274 stored elements in Compressed Sparse Row format>

**Q6**.
- On the one side, find what movies did the userId 4 rate?

- On the other side, what is the value of `ratings_matrix` for:
    - userId = 4 and movieId=1
    - userId = 4 and movieId=2
    - userId = 4 and movieId=21
    - userId = 4 and movieId=32
    - userId = 4 and movieId=126

Conclude on the values signification in `ratings_matrix`

In [21]:
ratings[ratings.userId==4]

,userId,movieId,rating,timestamp,date_time
300,4,21,3.0,986935199,2001-04-10 20:39:59
301,4,32,2.0,945173447,1999-12-14 12:10:47
302,4,45,3.0,986935047,2001-04-10 20:37:27
303,4,47,2.0,945173425,1999-12-14 12:10:25
304,4,52,3.0,964622786,2000-07-26 14:46:26
...,...,...,...,...,...
511,4,4765,5.0,1007569445,2001-12-05 16:24:05
512,4,4881,3.0,1007569445,2001-12-05 16:24:05
513,4,4896,4.0,1007574532,2001-12-05 17:48:52
514,4,4902,4.0,1007569465,2001-12-05 16:24:25


In [24]:
for mid in [1,2,21,32,126]:
    print("User id",4)
    print("User index",uid_to_idx[4])
    print("Movie id",mid)
    print("Movie index",mid_to_idx[mid])
    print("Matrix value (for user 4) is:",ratings_matrix[uid_to_idx[4], mid_to_idx[mid]])
    print("")

User id 4
User index 3
Movie id 1
Movie index 0
Matrix value (for user 4) is: 0.0

User id 4
User index 3
Movie id 2
Movie index 446
Matrix value (for user 4) is: 0.0

User id 4
User index 3
Movie id 21
Movie index 273
Matrix value (for user 4) is: 1.0

User id 4
User index 3
Movie id 32
Movie index 274
Matrix value (for user 4) is: 1.0

User id 4
User index 3
Movie id 126
Movie index 279
Matrix value (for user 4) is: 1.0



**Q7**. Now that you have a `ratings_matrix` in the correct format, let's save it in pickle format:
- Create a variable `dst_dir` corresponding to the path of the folder `data/netflix` located at the root of the repository
- **Verify that this is the correct path**
- Save the ratings_matrix in pickle (as `ratings_matrix.pkl`) in this corresponding directory

In [25]:
directory = "./data"
import pickle


In [26]:
pickle.dump(ratings_matrix, open(directory + "/ratings_matrix.pkl", "wb"))

**Q6**. Save also all mappings objects into pickle (`idx_to_mid`, `mid_to_idx`, `uid_to_idx`, `idx_to_uid`) as it will be useful for later.


In [29]:
pickle.dump(ratings_matrix, open(directory + "/idx_to_mid.pkl", "wb"))
pickle.dump(ratings_matrix, open(directory + "/mid_to_idx.pkl", "wb"))
pickle.dump(ratings_matrix, open(directory + "/uid_to_idx.pkl", "wb"))
pickle.dump(ratings_matrix, open(directory + "/idx_to_uid.pkl", "wb"))


In [28]:
pickle.dump(movies, open(directory + "/movies.pkl", "wb"))

Up to next challenge now! 🍿